In [1]:
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64
import json

In [2]:
import sys
sys.path.append('..')
import classified

In [3]:
api_passphrase = classified.PASSPHRASE
api_key = classified.KEY
api_secret = classified.SECRET

session = Session()

In [4]:
def auth(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": "2"
    }
    return headers

In [65]:
def getBalance(method, endpoint):
    url = 'https://api.kucoin.com' + endpoint
    auth(method, endpoint)
    r = session.get(url, headers=headers)
    data = r.json()['data']
    df = pd.DataFrame.from_dict(data)
    print(r.status_code)
    print(df)

In [14]:
def placeOrder(method, endpoint, side, symbol, price, size):
    url = 'https://api.kucoin.com/api/v1/orders'
    auth(method, endpoint)
    data = {
            "clientOid": "01", 
            "side": side, 
            "symbol": symbol, 
            "price": price,
            "size": size
            }
    data_json = json.dumps(data)
    response = session.post(url, headers=headers, data=data_json)
    print(response.json())

In [15]:
placeOrder('post', '/api/v1/orders', "buy", "ETH-USDT", "1195.15", "0.01")

{'code': '400100', 'msg': 'validation.createOrder.clientOidDuplicated'}
